In [1]:
import django
import sys
import os
sys.path.append('..')

from django.conf import settings
django.setup()

from campfin_data.models import *
import locale
locale.setlocale(locale.LC_ALL, 'en_US')
from django.db.models import Q, Sum, Count

In [2]:
# Count how many donations have clusters, and how many don't
print ScrapedContribution.objects.filter(dedupe_cluster__isnull=False).count(), 'clustered'
print ScrapedContribution.objects.filter(dedupe_cluster__isnull=True).count(), 'unclustered'

0 clustered
481453 unclustered


In [3]:
def translate_name(name):
    mapping = {
        'BRING BACK OUR $30 CAR TABS-VWMC-2016': 'Bring Back Our $30 Car Tabs',
        'KING CURTIS P': 'Curtis King',
        'JUDICIAL INTEGRITY WA PAC': 'Judicial Integrity Washington PAC',
        'TRUMP, DONALD J. / MICHAEL R. PENCE ': 'Donald Trump',
        'BRYANT WILLIAM L': 'Bill Bryant',
        'PIKE LIZ S': 'Liz Pike',
        'ZEMPEL GREGORY L': 'Greg Zempel',
        'DEWOLF DAVID K': 'David DeWolf',
        'REAGAN FUND': 'Reagan Fund',
        'MCMORRIS RODGERS, CATHY': 'Cathy McMorris Rodgers',
        'ROACH DANIEL N': 'Dan Roach',
        'MANWELLER MATHEW S': 'Mat Manweller',
        'WA ST REPUB PARTY EXEMPT': 'WA Republican Party',
        'LARSON DAVID A': 'David Larson',
        'SENATE REPUB CAMP COMM': 'Senate Republicans PAC',
        'HERRERA BEUTLER, JAIME': 'Jaime Herrera Beutler',
        'QUIRING EILEEN J': 'Eileen Quiring',
        'WILSON LYNDA D': 'Lynda Wilson',
        'WA ST REPUB PARTY NON EXEMPT': 'WA Republican Party',
        'WYMAN KIMBERLEY M': 'Kim Wyman',
        'ROACH PAM': 'Pam Roach',
        'GERHARDT LEWIS M JR': 'Lewis Gerhardt',
        'KILMER, DEREK': 'Derek Kilmer',
        'KRAFT VICKI C': 'Vicki Kraft',
        'HARRIS PAUL L': 'Paul Harris',
        'MCBURNEY PATRICK D JR': 'Patrick McBurney',
        'FORTUNATO PHILIP D': 'Philip Fortunato',
        'CRAIN CAROLYN I': 'Carolyn Crain',
        'WILSON LYNDA D': 'Lynda Wilson',
        'MILOSCIA MARK A': 'Mark Miloscia',
        'WAITE MICHAEL R': 'Michael Waite'
    }
    if name in mapping:
        return mapping[name]
    return name

contribs = ScrapedContribution.objects.filter(
    Q(donor__icontains='fisher, ken') | Q(donor__icontains='fisher ken') |
    Q(donor__icontains='holland, clyde') | Q(donor__icontains='holland clyde')).values(
    'donor', 'city', 'state', 'filing_committee__filer_id').annotate(sum=Sum('amount'))

print contribs.count(), 'contributions'

mapping = {
    'Ken Fisher': [],
    'Clyde Holland': []
}

meta_mapping = {
    'Ken Fisher': 'Fisher',
    'Fisher Investments': 'Fisher',
    'Clyde Holland': 'Holland',
    'Holland Partner Group': 'Holland'
}

for contrib in contribs:
    donor_slug = '-'.join([contrib['donor'], contrib['city'], contrib['state']])
    if contrib['donor'].find('FISHER, KEN') != -1 or contrib['donor'].find('FISHER KEN') != -1:
        mapping['Ken Fisher'].append(donor_slug)
    elif contrib['donor'].find('FISHER INVEST') != -1:
        mapping['Fisher Investments'].append(donor_slug)
    elif contrib['donor'].find('HOLLAND PARTNER') != -1:
        mapping['Holland Partner Group'].append(donor_slug)
    elif contrib['donor'].find('HOLLAND, CLYDE') != -1 or contrib['donor'].find('HOLLAND CLYDE') != -1:
        mapping['Clyde Holland'].append(donor_slug)
    else:
        print 'Uncategorized slug', donor_slug

nodes = {}
edges = []

output = {
    'nodes': [],
    'links': []
}

for contrib in contribs:
    donor_slug = '-'.join([contrib['donor'], contrib['city'], contrib['state']])
    
    entity = ''
    for key in mapping:
        for slug in mapping[key]:
            if slug == donor_slug:
                entity = key
                break
    cmte_slug = contrib['filing_committee__filer_id']
    if cmte_slug == 'WASTRPN188':
        cmte_slug = 'WASTRPE188'

    if entity not in nodes:
        # donations = ScrapedContribution.objects.filter(dedupe_cluster__slug=dedupe_slug)
        nodes[entity] = {
            'name': entity,
            'id': len(output['nodes']),
            'total': 0.,
            'type': 'donor'
        }
        #output['nodes'].append({'name': entity})
    nodes[entity]['total'] += float(contrib['sum'])
    
    cmte = ScrapedCommittee.objects.get(filer_id=cmte_slug)
    name = cmte.name
    if cmte.party == 'R':
        if cmte.office == 'STATE REPRESENTATIVE':
            name = 'State House Republican Candidates'
            cmte_slug = 'statehouserepublicans'
        elif cmte.office == 'STATE SENATOR':
            name = 'State Senate Republican Candidates'
            cmte_slug = 'statesenaterepublicans'

    if cmte_slug not in nodes:        
        nodes[cmte_slug] = {
            'slug': cmte_slug,
            'name': name,
            'received_from': {},
            'type': 'committee',
            'total': 0.
        }
    if entity not in nodes[cmte_slug]['received_from']:
        nodes[cmte_slug]['received_from'][entity] = 0.
    nodes[cmte_slug]['received_from'][entity] += float(contrib['sum'])
    nodes[cmte_slug]['total'] += float(contrib['sum'])

for node in nodes:
    if nodes[node]['total'] >= 1.:
        nodes[node]['id'] = len(output['nodes'])
        output['nodes'].append({'name': translate_name(nodes[node]['name'])})
    
print len(nodes)

supplements = {
    #'Bernie Sanders': {'fisher': 35., 'holland': 0. },
    'McMorris Rodgers American Dream Project': {'fisher': 5000., 'holland': 5000. },
    'National Republican Congressional Committee': {'fisher': 5000., 'holland': 1000. },
    'Republican National Committee': {'fisher': 19600., 'holland': 33400. },
    'Trump Victory Fund': {'fisher': 25000., 'holland': 94600. },
    'Washington State Republican Party (FEC)': {'fisher': 22000., 'holland': 22000. },
    #'Michael Bennet': {'fisher': 0., 'holland': 2700. },
    'Ben Carson': {'fisher': 0., 'holland': 2700. },
    #'Kevin McCarthy': {'fisher': 0., 'holland': 2700. },
    'Marco Rubio': {'fisher': 0., 'holland': 2634. },
    #'McCarthy Victory Fund': {'fisher': 0., 'holland': 2700. },
    'National Multifamily Housing Council PAC': {'fisher': 0., 'holland': 5000. },
    #'Bruce Poliquin': {'fisher': 0., 'holland': 1000. },
    'Right to Rise USA': {'fisher': 0., 'holland': 25000. },
    #'Ron Wyden': {'fisher': 0., 'holland': 5400. }
}

for supplement in supplements:
    cmte_slug = supplement.replace(' ', '')
    nodes[cmte_slug] = {
        'slug': cmte_slug,
        'name': supplement,
        'received_from': {},
        'type': 'committee',
        'total': 0.
    }
    
    for donor in ['fisher', 'holland']:
        if supplements[supplement][donor] > 0.:
            if donor == 'fisher':
                key = 'Ken Fisher'
            else:
                key = 'Clyde Holland'
            nodes[cmte_slug]['received_from'][key] = supplements[supplement][donor]
    nodes[cmte_slug]['id'] = len(output['nodes'])
    output['nodes'].append({'name': supplement})



import json

"""
intermediate_groups = ['SEIU', 'Fisher', 'Holland']
intermediate_groups = ['SEIU']


for key in intermediate_groups:
    nodes[key] = {
        'name': key,
        'id': len(output['nodes']),
        'type': 'intermediate'
    }
    output['nodes'].append({'name': key})
"""

for node in nodes:
    if nodes[node]['type'] == 'committee':
        #intermediates = {}

        for donor in nodes[node]['received_from']:
            try:
                output['links'].append({
                    'source': nodes[donor]['id'],
                    'target': nodes[node]['id'],
                    'value': nodes[node]['received_from'][donor]
                })
            except:
                continue
    """
            donor_entity = meta_mapping[donor]
            if donor_entity not in intermediates:
                intermediates[donor_entity] = {
                    'id': nodes[donor_entity]['id'],
                    'amount': 0.
                }
            intermediates[donor_entity]['amount'] += float(nodes[node]['received_from'][donor])

        for key in intermediates:
            output['links'].append({
                'source': intermediates[key]['id'],
                'target': nodes[node]['id'],
                'value': intermediates[key]['amount']
            })
        """
    """
    if nodes[node]['type'] == 'donor':
        print nodes[node]
        output['links'].append({
            'source': nodes[node]['id'],
            'target': nodes[meta_mapping[node]]['id'],
            'value': nodes[node]['total']
        })
    """
            
"""
for edge in edges:
    output['links'].append({
        'source': edge['donor'],
        'target': edge['recip'],
        'value': edge['amount']
    })
"""

with open('/home/aepton/code/campfin/data/donors_sankey_fisher_holland_supplemented.json', 'w+') as fh:
    fh.write(json.dumps(output))

with open('/home/aepton/code/dataviz/graphviz/donors_sankey_fisher_holland_supplemented.json', 'w+') as fh:
    fh.write(json.dumps(output))

52 contributions
25


IOError: [Errno 2] No such file or directory: '/home/aepton/code/campfin/data/donors_sankey_fisher_holland_supplemented.json'

In [4]:
def translate_name(name):
    mapping = {
        'YES FOR HOMES': 'Yes for Homes',
        'MYVOTECOUNTS': 'My Vote Counts',
        'KENNEDY FUND': 'Kennedy Fund',
        'CAMPAIGN TO PREVENT FRAUD AND PROTECT SENIORS': 'Campaign to Prevent Fraud and Protect Seniors',
        'HARRY TRUMAN FUND': 'Harry Truman Fund',
        'OUR WASHINGTON': 'Our Washington',
        'ONE AMERICA VOTES': 'One America Votes',
        'FUSE VOTES': 'FUSE Votes',
        'FERGUSON ROBERT W': 'Robert Ferguson',
        'YES ON I-125 COMM': 'Yes on I-125',
        'WA ST DEMO CENT COMM EXEMPT': 'WA Democratic Party',
        'WORKING FAMILIES PARTY': 'Working Families Party',
        'INSLEE JAY R': 'Jay Inslee',
        'RAISE UP WA': 'Raise Up Washington',
        'MASS TRANSIT NOW'
    }
    if name in mapping:
        return mapping[name]
    return name

contribs = ScrapedContribution.objects.filter(
    Q(donor__icontains='seiu') | Q(donor__icontains='service employee')).values(
    'donor', 'city', 'state', 'filing_committee__filer_id', 'filing_committee__name').annotate(sum=Sum('amount'))

print contribs.count(), 'contributions'

mapping = {
    'SEIU Local 1199 (nurses)': [],
    'SEIU Local 925 (education)': [],
    'SEIU Local 775 (healthcare)': [],
    'SEIU Local 6 (janitors)': [],
    'SEIU Local 1948 (schools)': [],
    'SEIU Healthcare': [],
    'SEIU (other locals, state, national)': []
}

meta_mapping = {
    'SEIU Local 1199 (nurses)': 'SEIU',
    'SEIU Local 925 (education)': 'SEIU',
    'SEIU Local 775 (healthcare)': 'SEIU',
    'SEIU Local 6 (janitors)': 'SEIU',
    'SEIU Local 1948 (schools)': 'SEIU',
    'SEIU Healthcare': 'SEIU',
    'SEIU (other locals, state, national)': 'SEIU'
}

for contrib in contribs:
    donor_slug = '-'.join([contrib['donor'], contrib['city'], contrib['state']])
    if contrib['donor'].find('1199') != -1 or contrib['donor'].find('119NW') != -1:
        mapping['SEIU Local 1199 (nurses)'].append(donor_slug)
    elif contrib['donor'].find('925') != -1:
        mapping['SEIU Local 925 (education)'].append(donor_slug)
    elif contrib['donor'].find('775') != -1 or contrib['donor'].find('755') != -1:
        mapping['SEIU Local 775 (healthcare)'].append(donor_slug)
    elif contrib['donor'].find('6') != -1:
        mapping['SEIU Local 6 (janitors)'].append(donor_slug)
    elif contrib['donor'].find('1948') != -1:
        mapping['SEIU Local 1948 (schools)'].append(donor_slug)
    elif contrib['donor'].find('SEIU') != -1 and contrib['city'] == 'RENTON':
        mapping['SEIU Healthcare'].append(donor_slug)
    elif contrib['donor'].find('SEIU') != -1 or contrib['donor'].find('SERVICE EMPLOYEES') != -1:
        mapping['SEIU (other locals, state, national)'].append(donor_slug)
    else:
        print 'Uncategorized slug', donor_slug

nodes = {}
edges = []

output = {
    'nodes': [],
    'links': []
}

for contrib in contribs:
    donor_slug = '-'.join([contrib['donor'], contrib['city'], contrib['state']])
    
    entity = ''
    for key in mapping:
        for slug in mapping[key]:
            if slug == donor_slug:
                entity = key
                break
    cmte_slug = contrib['filing_committee__filer_id']

    if entity not in nodes:
        # donations = ScrapedContribution.objects.filter(dedupe_cluster__slug=dedupe_slug)
        nodes[entity] = {
            'name': entity,
            #'id': len(output['nodes']),
            'total': 0.,
            'type': 'donor'
        }
        #output['nodes'].append({'name': entity})
    nodes[entity]['total'] += float(contrib['sum'])
    
    cmte = ScrapedCommittee.objects.get(filer_id=cmte_slug)
    name = cmte.name
    if cmte.party == 'D':
        if cmte.office == 'STATE REPRESENTATIVE':
            name = 'State House Democratic Candidates'
            cmte_slug = 'statehousedemocrats'
        elif cmte.office == 'STATE SENATOR':
            name = 'State Senate Democratic Candidates'
            cmte_slug = 'statesenatedemocrats'

    if name.find('SERVICE EMPLOYEE') != -1:
        continue
    if cmte_slug not in nodes:
        nodes[cmte_slug] = {
            'slug': cmte_slug,
            'name': name,
            'received_from': {},
            'type': 'committee',
            'total': 0.
        }
    if entity not in nodes[cmte_slug]['received_from']:
        nodes[cmte_slug]['received_from'][entity] = 0.
    nodes[cmte_slug]['received_from'][entity] += float(contrib['sum'])
    nodes[cmte_slug]['total'] += float(contrib['sum'])

for node in nodes:
    if nodes[node]['total'] >= 10000.:
        nodes[node]['id'] = len(output['nodes'])
        output['nodes'].append({'name': translate_name(nodes[node]['name'])})
    
print len(nodes)

import json

"""
intermediate_groups = ['SEIU']


for key in intermediate_groups:
    nodes[key] = {
        'name': key,
        'id': len(output['nodes']),
        'type': 'intermediate'
    }
    output['nodes'].append({'name': key})
"""

for node in nodes:
    if nodes[node]['type'] == 'committee':
        #intermediates = {}

        for donor in nodes[node]['received_from']:
            try:
                output['links'].append({
                    'source': nodes[donor]['id'],
                    'target': nodes[node]['id'],
                    'value': nodes[node]['received_from'][donor]
                })
            except:
                continue
    """
            donor_entity = meta_mapping[donor]
            if donor_entity not in intermediates:
                intermediates[donor_entity] = {
                    'id': nodes[donor_entity]['id'],
                    'amount': 0.
                }
            intermediates[donor_entity]['amount'] += float(nodes[node]['received_from'][donor])

        for key in intermediates:
            output['links'].append({
                'source': intermediates[key]['id'],
                'target': nodes[node]['id'],
                'value': intermediates[key]['amount']
            })
        """
    """
    if nodes[node]['type'] == 'donor':
        print nodes[node]
        output['links'].append({
            'source': nodes[node]['id'],
            'target': nodes[meta_mapping[node]]['id'],
            'value': nodes[node]['total']
        })
    """
            
"""
for edge in edges:
    output['links'].append({
        'source': edge['donor'],
        'target': edge['recip'],
        'value': edge['amount']
    })
"""

with open('/home/aepton/code/campfin/data/donors_sankey_seiu.json', 'w+') as fh:
    fh.write(json.dumps(output))

with open('/home/aepton/code/dataviz/graphviz/donors_sankey_seiu.json', 'w+') as fh:
    fh.write(json.dumps(output))

SyntaxError: invalid syntax (<ipython-input-4-22b674ceac86>, line 18)

In [5]:
print BallotCandidate.objects.all().count(), 'candidates'
print BallotDistrict.objects.all().count(), 'districts'
print BallotDistrictType.objects.all().count(), 'district types'
print BallotDistrictDistrictMap.objects.all().count(), 'district-district maps'
print BallotDistrictPrecinctMap.objects.all().count(), 'district-precinct maps'
print BallotElection.objects.all().count(), 'elections'
print BallotOffice.objects.all().count(), 'offices'
print BallotPrecinct.objects.all().count(), 'precincts'
print BallotRaceSummary.objects.all().count(), 'race summaries'

31979 candidates
13725 districts
23 district types
759 district-district maps
173310 district-precinct maps
63 elections
20766 offices
11808 precincts
62518 race summaries


In [6]:
# SEIU historical contribution processing

contribs = PDCContribution.objects.exclude(is_superseded=True).filter(
    Q(donor__full_name__icontains='seiu') | Q(donor__full_name__icontains='service employee'))

groups = {
    'house': {},
    'senate': {}
}

for contrib in contribs:
    cmte = contrib.filer_committee
    cmte_year = PDCCommitteeYear.objects.filter(committee=cmte, year=contrib.rcpt_date.year)
    if cmte_year:
        if not cmte_year[0].jurisdiction:
            key = cmte.name
        else:
            if cmte_year[0].jurisdiction.find('LEG DISTRICT') != -1:
                if cmte_year[0].jurisdiction.find('HOUSE') != -1:
                    key = 'house'
                else:
                    key = 'senate'
            else:
                key = cmte.name
    else:
        key = cmte.name
    
    if key not in groups:
        groups[key] = {}

    if contrib.rcpt_date.year not in groups[key]:
        groups[key][contrib.rcpt_date.year] = 0.
    groups[key][contrib.rcpt_date.year] += float(contrib.amount)

print groups

{u'KITSAP COMM FOR DEMOCRACY': {2014: 91000.0}, u'CHILDRENS CAMPAIGN FUND': {2008: 250.0, 2014: 200.0, 2006: 500.0, 2007: 300.0}, u'KASTAMA JAMES M': {1996: 300.0}, u'SERVICE EMPLOYEES INTL UNION WA ST COUNCIL PAC': {2016: 35000.0, 2012: 1092000.0, 2013: 146000.0, 2014: 672005.0, 2015: 427000.0}, u'BONLENDER RONALD J': {2007: 150.0}, u'CIT FOR SHORELINE SCHOOLS': {2006: 300.0}, u'MANN THOMAS D': {2013: 250.0}, u'HUFF SHERRIL D': {2009: 800.0}, u'WALDREF AMBER C': {2009: 750.0, 2013: 1400.0}, u'35TH DIST CIT FOR INTEGRITY IN GOVT': {2014: 10000.0}, u'NO ON I 960': {2007: 257417.16}, u'EDMONDS CAROLYN A': {2001: 250.0, 2005: 500.0}, u'QUALIFIED LEADERSHIP FOR SEATTLE': {2009: 3500.0}, u'REYKDAL CHRIS P': {2013: 900.0}, u'LITTLE RONNIE B': {1994: 100.0}, u'VOTER EDUCATION COMM': {2008: 44000.0, 2009: 2500.0, 2011: 2000.0, 2004: 12889.36, 2005: 50000.0}, u'MUMM CANDACE M': {2013: 2150.0}, u'WASHINGTONIANS FOR EDUCATION HEALTH & TAX RELIEF': {2010: 2580754.5899999994}, u'MARSHALL ROCKEY L':

In [7]:
output_rows = []

for group in sorted(groups.keys(), key=lambda k: len(groups[k]), reverse=True):
    output = {'name': group}
    for year in range(1994, 2016):
        if year not in groups[group]:
            output[year] = 0.
        else:
            output[year] = groups[group][year]
    output_rows.append(output)

In [8]:
from csv import DictWriter

with open('/home/aepton/code/campfin/data/seiu_yearly.csv', 'w+') as fh:
    writer = DictWriter(fh, sorted(output_rows[0].keys()))
    writer.writeheader()
    writer.writerows(output_rows)

IOError: [Errno 2] No such file or directory: '/home/aepton/code/campfin/data/seiu_yearly.csv'

In [ ]:
# SEIU current-cycle contribution processing

contribs = ScrapedContribution.objects.filter(Q(donor__icontains='seiu') | Q(donor__icontains='service employee'))

groups = {
    'house': {},
    'senate': {}
}

for contrib in contribs:
    cmte = contrib.filing_committee
    if not cmte.office:
        key = cmte.name
    else:
        if cmte.office == 'STATE SENATOR':
            key = 'senate'
            if contrib.date.year == 2015:
                print 'senate', contrib.amount, contrib.date, contrib.filing_committee.name, contrib.donor
        elif cmte.office == 'STATE REPRESENTATIVE':
            key = 'house'
            if contrib.date.year == 2015:
                print 'house', contrib.amount, contrib.date, contrib.filing_committee.name, contrib.donor
        else:
            key = cmte.name
    
    if key not in groups:
        groups[key] = {}

    if contrib.date.year not in groups[key]:
        groups[key][contrib.date.year] = 0.
    groups[key][contrib.date.year] += float(contrib.amount)

print 'Found %d groups' % len(groups)

output_rows = []

for group in sorted(groups.keys(), key=lambda k: len(groups[k]), reverse=True):
    output = {'name': group}
    for year in range(2015, 2017):
        if year not in groups[group]:
            output[year] = 0.
        else:
            output[year] = groups[group][year]
    output_rows.append(output)

from csv import DictWriter

with open('/home/aepton/code/campfin/data/seiu_yearly_scraped.csv', 'w+') as fh:
    writer = DictWriter(fh, sorted(output_rows[0].keys()))
    writer.writeheader()
    writer.writerows(output_rows)